In [16]:
import ijson
import numpy as np
import json
from decimal import Decimal
import json
import time
import torch
import warnings
import numpy as np
from IPython.display import clear_output
import pennylane as qml
from sklearn.model_selection import train_test_split
import pennylane as qml
import numpy as np
import torch
from scipy.linalg import expm
import numpy as np
from scipy.linalg import qr

import array_to_latex

The transverse and longitudinal impact parameters (IP), d0 and z0, are defined as the point
to the closest approach of the trajectory of a track to the primary vertex in the transverse
plane and z-direction, respectively

## Robustness test (Caso Básico)


We are going to validate the robustness of the model by replicating the results mathematically and graphically, checking later the similarity between results.

$\theta_1$ = $\frac{\pi}{2}$, $\theta_2$ = $\frac{\pi}{4}$, $\phi_1$ = $\frac{2\pi}{3}$ and $\phi_2$ = $\frac{3\pi}{4}$

Dados estos 4 angulos podemos obtener un estado de Majora tal que:
$$

\ket{\psi_g} = \Gamma 
\begin{pmatrix}
e^{i \phi_1} \sin \frac{\theta_1}{2} \cos \frac{\theta_2}{2} 
+ e^{i \phi_2} \cos \frac{\theta_1}{2} \sin \frac{\theta_2}{2} \\[6pt]
\sqrt{2}\, e^{i(\phi_1+\phi_2)} \sin \frac{\theta_1}{2} \sin \frac{\theta_2}{2}
\end{pmatrix}
$$

$$
\Gamma = \sqrt{2}\,\Big[ 3 + \cos\theta_1 \cos\theta_2 
+ \sin\theta_1 \sin\theta_2 \cos(\phi_1 - \phi_2) \Big]^{-\tfrac{1}{2}}
$$

$$
\theta_i \in [0,\pi], 
\quad \phi_i \in [0,2\pi], 
\quad i = 1,2.
$$

$$
\Gamma : (0.7441)
$$

$$
\text{State: } 
\begin{bmatrix}
 0.6874 + 0.0j \\
-0.1007 - 0.3757j \\
-0.2466 + 0.1424j
\end{bmatrix}
$$

$$
\text{Norma: } 0.8396
$$

$$
\text{Estado normalizado: }
\begin{bmatrix}
 0.8188 + 0.0j \\
-0.1199 - 0.4475j \\
-0.2937 + 0.1696j
\end{bmatrix}
$$

In [160]:
theta1 = torch.pi / torch.tensor(2)
theta2 = torch.pi / torch.tensor(4)
phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)

def inicializing_qutrit_state(theta1, theta2, phi1, phi2):
    Gamma= 0
    a0 = 0
    a1 = 0
    a2 = 0

    Gamma = torch.sqrt(torch.tensor(2.0)) * (torch.tensor(3.0) + torch.cos(theta1)*torch.cos(theta2) + torch.sin(theta1)*torch.sin(theta2)*torch.cos(phi1 - phi2))**(torch.tensor(-0.5))

    # Calcular las amplitudes
    a0 = (torch.sqrt(torch.tensor(2.0)) * torch.cos(theta1/2) * torch.cos(theta2/2)).item()
    a1 = (torch.exp(1j * phi1) * torch.sin(theta1/2) * torch.sin(theta2/2) + torch.cos(theta1/2) * torch.sin(theta2/2) * torch.exp(1j * phi2)).item()
    a2 = (torch.sqrt(torch.tensor(2.0)) * torch.exp(1j * (phi1 + phi2)) * torch.sin(theta1/2) * torch.sin(theta2/2)).item()


    # vecctor de estado
    state = Gamma * torch.tensor([a0, a1, a2], dtype=torch.cdouble)
    print('Gamma:', Gamma)
    print('State:', state)


    print('Norma: ',torch.linalg.norm(state))
    # NoOrmalizar 
    state = state / torch.linalg.norm(state)
    print('Estado normalizado:', state)
    

    return state.detach().clone().numpy()

A = inicializing_qutrit_state(theta1, theta2, phi1, phi2)

Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)


In [9]:
print(A)

[ 0.81875611+0.j         -0.11990404-0.44748807j -0.2937038 +0.16956992j]


In [124]:
def unitary_from_state(psi):
    """
    psi: vector complejo normalizado de dimension 3 (qutrit)
    retorna: unitaria 3x3 cuyo primer columna es psi
    """
    psi = psi / np.linalg.norm(psi)  # por seguridad
    
    a1 = torch.tensor([0.555,0, 0.555], dtype=torch.cdouble)
    a2 = torch.tensor([0.555,0.555, 0], dtype=torch.cdouble)
    mat = np.column_stack([psi, a1 , a2])
    #print(mat)

    
    # QR para ortonormalizar columnas
    Q, R = qr(mat)
    print(np.round(Q, 4))
    # Ajustamos fase de la primera columna para que coincida exactamente con psi
    phase = np.vdot(psi, Q[:,0])
    Q[:,0] = Q[:,0] * (phase/abs(phase)).conj()
    
    return Q

Q = unitary_from_state(A)



[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]


### Usando el proceso de Gram–Schmidt podemos crear nuestra unitaria donde el estado resultado de la codificación inicial se encuentre en la columna 1 

Consideremos el **proceso de Gram–Schmidt** aplicado a las columnas de la matriz de rango completo
$$
A = \begin{bmatrix} \mathbf{a}_1 & \cdots & \mathbf{a}_n \end{bmatrix},
$$
con producto interno
$$
\langle \mathbf{v}, \mathbf{w} \rangle = \mathbf{v}^\top \mathbf{w} 
\quad (\text{o } \langle \mathbf{v}, \mathbf{w} \rangle = \mathbf{v}^\dagger \mathbf{w} \text{ en el caso complejo}).
$$

Definimos la proyección:
$$
\operatorname{proj}_{\mathbf{u}} \mathbf{a} = \frac{\langle \mathbf{u}, \mathbf{a} \rangle}{\langle \mathbf{u}, \mathbf{u} \rangle}\,\mathbf{u}.
$$

Entonces:
$$
\mathbf{u}_1 = \mathbf{a}_1,
$$
$$
\mathbf{u}_2 = \mathbf{a}_2 - \operatorname{proj}_{\mathbf{u}_1} \mathbf{a}_2,
$$
$$
\mathbf{u}_3 = \mathbf{a}_3 - \operatorname{proj}_{\mathbf{u}_1} \mathbf{a}_3 - \operatorname{proj}_{\mathbf{u}_2} \mathbf{a}_3,
$$
$$
\vdots
$$
$$
\mathbf{u}_k = \mathbf{a}_k - \sum_{j=1}^{k-1} \operatorname{proj}_{\mathbf{u}_j} \mathbf{a}_k.
$$

Finalmente, los vectores ortonormales se obtienen como:
$$
\mathbf{e}_1 = \frac{\mathbf{u}_1}{\|\mathbf{u}_1\|}, \quad
\mathbf{e}_2 = \frac{\mathbf{u}_2}{\|\mathbf{u}_2\|}, \quad
\mathbf{e}_3 = \frac{\mathbf{u}_3}{\|\mathbf{u}_3\|}, \quad
\ldots \quad
\mathbf{e}_k = \frac{\mathbf{u}_k}{\|\mathbf{u}_k\|}.
$$

Por lo tanto nuestro vector unitario con el estado inicial que queremos codificar en la primera columna quedaria tal que:
$$
U = Q = [ e_1, e_2, e_3]
$$



In [119]:
e1 = 0
e2 = 0
e3 = 0
u2 = 0
u3 = 0


print('Matriz Q:')
print(np.round(Q,4))
print('-----------------------------------------------------')
e1 = np.array([0.8187, -0.1199-0.4474j, -0.2937+0.1696j])
a2 = np.array([0.555,0,0.555])
a3 = np.array([0.555,0.555,0])

print('e1: ',np.round(e1,4))
print('-----------------------------------------------------')

proj = (np.vdot(e1, a2)/np.vdot(e1, e1)) * e1
u2 = a2 - proj
e2 = u2/np.linalg.norm(u2)

print('e2: ',np.round(e2,4))

print('-----------------------------------------------------')
e3 = 0
u3 = a3 - ((np.vdot(e1, a3)/np.vdot(e1, e1)) * e1) - ((np.vdot(u2, a3)/np.vdot(u2, u2)) * u2)
e3 = u3/np.linalg.norm(u3)
print('e3: ',np.round(e3,4))


Matriz Q:
[[ 0.8188-0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [-0.1199-0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [-0.2937+0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
-----------------------------------------------------
e1:  [ 0.8187+0.j     -0.1199-0.4474j -0.2937+0.1696j]
-----------------------------------------------------
e2:  [0.4378+0.1067j 0.1066+0.1648j 0.8643-0.1067j]
-----------------------------------------------------
e3:  [ 0.1654-0.315j  0.8048+0.315j -0.1654+0.315j]


Verificado que la creación de la unitaria funciona:

$$
\ket{\psi} = U\ket{0} = \begin{bmatrix}
0.8188 - 0.0000i & -0.4378 - 0.1066i & 0.1654 - 0.3150i \\
-0.1199 - 0.4475i & -0.1066 - 0.1648i & 0.8048 + 0.3150i \\
-0.2937 + 0.1696i & -0.8643 + 0.1066i & -0.1654 + 0.3150i
\end{bmatrix}\begin{bmatrix}
1\\
0 \\
0
\end{bmatrix} = \begin{bmatrix}
0.8188 - 0.0000i \\
-0.1199 - 0.4475i \\
-0.2937 + 0.1696i
\end{bmatrix}
$$

# Cirucito 1 para comprobar que el encoding es robusto

In [270]:
wires = [0]
dev = qml.device("default.qutrit", wires=wires)  

theta1 = torch.pi / torch.tensor(2)
theta2 = torch.pi / torch.tensor(4)
phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)

@qml.qnode(dev)
def circuit():
    initial_state = inicializing_qutrit_state(theta1, theta2, phi1, phi2)
    u = unitary_from_state(initial_state)
    qml.QutritUnitary(u, wires=0)
    return qml.state()

estado_initial = circuit()
print('Estado inicial:')
print(np.round(estado_initial,4))
print(estado_initial.shape)



Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
Estado inicial:
[ 0.8188-0.j     -0.1199-0.4475j -0.2937+0.1696j]
(3,)


Vale el encoding es coherente

# Comprobamos que las rotaciones son coherentes

Las matrices de rotación son:

$$
\Sigma_1 = \frac{1}{\sqrt{2}}
\begin{bmatrix}
0 & 1 & 0 \\
1 & 0 & 1 \\
0 & 1 & 0
\end{bmatrix}, \quad
\Sigma_2 = \frac{1}{\sqrt{2}}
\begin{bmatrix}
0 & -i & 0 \\
i & 0 & -i \\
0 & i & 0
\end{bmatrix}, \quad
\Sigma_3 =
\begin{bmatrix}
1 & 0 & 0 \\
0 & 0 & 0 \\
0 & 0 & -1
\end{bmatrix}
$$

Para aplicar la rotación moviendo los dos puntos de la esfera de Majorana, aplicamos la siguiente fórmual que esra relacionada con la rotation de $\epsilon$ de los dos punto de Majorana sobre el eje $i$:
$$
U_i(\xi) = e^{\, i \xi \Sigma_i}
= I + (\cos \xi - 1)\,\Sigma_i^2 + i \sin \xi \,\Sigma_i
$$

In [217]:
Lambda = {
    1: torch.tensor([[0, 1, 0],
                 [1, 0, 0],
                 [0, 0, 0]], dtype=torch.cdouble),

    2: torch.tensor([[0, -1j, 0],
                 [1j, 0, 0],
                 [0, 0, 0]], dtype=torch.cdouble),

    3: torch.tensor([[1, 0, 0],
                 [0, -1, 0],
                 [0, 0, 0]], dtype=torch.cdouble),

    4: torch.tensor([[0, 0, 1],
                 [0, 0, 0],
                 [1, 0, 0]], dtype=torch.cdouble),

    5: torch.tensor([[0, 0, -1j],
                 [0, 0, 0],
                 [1j, 0, 0]], dtype=torch.cdouble),

    6: torch.tensor([[0, 0, 0],
                 [0, 0, 1],
                 [0, 1, 0]], dtype=torch.cdouble),

    7: torch.tensor([[0, 0, 0],
                 [0, 0, -1j],
                 [0, 1j, 0]], dtype=torch.cdouble),

    8: (1/torch.sqrt(torch.tensor(3.0))) * torch.tensor([[1, 0, 0],
                                  [0, 1, 0],
                                  [0, 0, -2]], dtype=torch.cdouble),
    0: torch.tensor([[1, 0, 0],
                 [0, 1, 0],
                 [0, 0, 1]], dtype=torch.cdouble)
}

# Spin-1 rotation generators (SO(3) ⊂ SU(3))
Sigma = {
    1: (1 / torch.sqrt(torch.tensor(2.0))) * torch.tensor([
        [0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]
    ], dtype=torch.cdouble),

    2: (1 / torch.sqrt(torch.tensor(2.0))) * torch.tensor([
        [0, -1j, 0],
        [1j, 0, -1j],
        [0, 1j, 0]
    ], dtype=torch.cdouble),

    3: torch.tensor([
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, -1]
    ], dtype=torch.cdouble)
}

def unitary_from_generator(generator_matrix, theta):
    if not torch.is_tensor(theta):
        theta = torch.tensor(theta, dtype=torch.cdouble)
    i = torch.tensor(1j, dtype=torch.cdouble)
    return Lambda[0] + (torch.cos(theta) - torch.tensor(1.0)) * generator_matrix @ generator_matrix + i * torch.sin(theta) * generator_matrix

theta1 = torch.pi / torch.tensor(2)
theta2 = torch.pi / torch.tensor(4)
phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)

estado_rotado_mat = 0
u_x = unitary_from_generator(Sigma[1], theta1)
u_y = unitary_from_generator(Sigma[2], theta2)
u_z = unitary_from_generator(Sigma[3], phi2)
print('U_x:')
print(np.round(u_x.clone().detach().numpy(),4))
print('U_y:')
print(np.round(u_y.clone().detach().numpy(),4))
print('U_z:')
print(np.round(u_z.clone().detach().numpy(),4))
print('-----------------------------------------------------')
estado_rotado_mat = u_x.clone().detach().numpy() @ estado_initial
estado_rotado_mat = u_y.clone().detach().numpy() @ estado_rotado_mat
estado_rotado_mat = u_z.clone().detach().numpy() @ estado_rotado_mat
print('-----------------------------------------------------')
print('Estado tras las rotaciones (matematiamente):')
print(np.round(estado_rotado_mat,4))

U_x:
[[ 0.5+0.j      0. +0.7071j -0.5+0.j    ]
 [ 0. +0.7071j  0. +0.j      0. +0.7071j]
 [-0.5+0.j      0. +0.7071j  0.5+0.j    ]]
U_y:
[[ 0.8536+0.j  0.5   +0.j  0.1464+0.j]
 [-0.5   +0.j  0.7071+0.j  0.5   +0.j]
 [ 0.1464+0.j -0.5   +0.j  0.8536+0.j]]
U_z:
[[-0.5-0.866j  0. +0.j     0. +0.j   ]
 [ 0. +0.j     1. +0.j     0. +0.j   ]
 [ 0. +0.j     0. +0.j    -0.5+0.866j]]
-----------------------------------------------------
-----------------------------------------------------
Estado tras las rotaciones (matematiamente):
[-0.2895-0.5832j -0.641 +0.3473j  0.1907+0.0906j]


Para $\theta_" = \pi/2$, $\theta_2 = \pi/4$ y $\phi_2 = 4\pi/3$

$$
U_x =
\begin{bmatrix}
0.5 & 0.7071 i & -0.5 \\
0.7071 i & 0 & 0.7071 i \\
-0.5 & 0.7071 i & 0.5
\end{bmatrix}, \quad
U_y =
\begin{bmatrix}
0.8536 & 0.5 & 0.1464 \\
-0.5 & 0.7071 & 0.5 \\
0.1464 & -0.5 & 0.8536
\end{bmatrix}, \quad
U_z =
\begin{bmatrix}
-0.5 - 0.866i & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & -0.5 + 0.866i
\end{bmatrix}
$$

$$
\ket{\psi} = \begin{bmatrix}
0.8188 - 0.0000i \\
-0.1199 - 0.4475i \\
-0.2937 + 0.1696i
\end{bmatrix}
$$

$$
U_z(\phi_2)U_y(\theta_2)U_x(\theta_1)\ket{\psi} \;=\;\begin{bmatrix}
-0.5 - 0.866i & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & -0.5 + 0.866i
\end{bmatrix} \cdot \begin{bmatrix}
0.8536 & 0.5 & 0.1464 \\
-0.5 & 0.7071 & 0.5 \\
0.1464 & -0.5 & 0.8536
\end{bmatrix} \cdot \begin{bmatrix}
0.5 & 0.7071 i & -0.5 \\
0.7071 i & 0 & 0.7071 i \\
-0.5 & 0.7071 i & 0.5
\end{bmatrix} \ket{\psi} = \begin{bmatrix}
-0.2895 - 0.5832i \\
-0.6410 + 0.3473i \\
0.1907 + 0.0906i
\end{bmatrix}
$$

In [230]:
wires = [0]
dev = qml.device("default.qutrit", wires=wires)  

theta1 = torch.pi / torch.tensor(2)
theta2 = torch.pi / torch.tensor(4)
phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)

print('Estado inicial:')
@qml.qnode(dev)
def circuit2():
    initial_state = inicializing_qutrit_state(theta1, theta2, phi1, phi2)
    u = unitary_from_state(initial_state)
    qml.QutritUnitary(u, wires=0)
    RX = unitary_from_generator(Sigma[1], theta1)
    RY = unitary_from_generator(Sigma[2], theta2)
    RZ = unitary_from_generator(Sigma[3], phi2)

    qml.QutritUnitary(RX, wires=0)
    qml.QutritUnitary(RY, wires=0)
    qml.QutritUnitary(RZ, wires=0)

    return qml.state()

estado_rotado = circuit2()
print('-----------------------------------------------------')
print('Estado tras las rotaciones:')
print(np.round(estado_rotado.clone().detach().numpy(),4))

Estado inicial:
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
-----------------------------------------------------
Estado tras las rotaciones:
[-0.2895-0.5832j -0.641 +0.3473j  0.1907+0.0906j]


---

---

---

# Robustness Test (Caso complejo)

## **Paso 1. Encoding**

$$
q_0 = \ket{\psi} = \begin{bmatrix}
0.8188 - 0.0000i \\
-0.1199 - 0.4475i \\
-0.2937 + 0.1696i
\end{bmatrix}
$$

$$
q_1 = \ket{\phi} = \begin{bmatrix}
0.8188 - 0.0000i \\
-0.1199 - 0.4475i \\
-0.2937 + 0.1696i
\end{bmatrix}
$$

$$
q_3^{\text{Ancilla}} = \ket{\Psi} = \begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix}
$$

Estado todal del sistema es:
$$
\ket{\psi} = 
\begin{bmatrix}
0.8188 - 0.0000i \\
-0.1199 - 0.4475i \\
-0.2937 + 0.1696i
\end{bmatrix},\quad
\ket{\phi} =
\begin{bmatrix}
0.8188 - 0.0000i \\
-0.1199 - 0.4475i \\
-0.2937 + 0.1696i
\end{bmatrix},\quad
\ket{\Psi} =
\begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix}
$$

$$
\ket{\text{Sistema}} = 
\Big( \ket{\psi} \otimes \ket{\phi} \Big) \otimes \ket{\Psi} =
\begin{bmatrix}
0.6704-0.j \\
0. +0.j \\
0. +0.j \\
-0.0982-0.3664j \\
0. -0.j \\
0. -0.j \\
-0.2405+0.1388j \\
-0. +0.j \\
-0. +0.j \\
-0.0982-0.3664j \\
0. -0.j \\
0. -0.j \\
-0.1859+0.1073j \\
-0. +0.j \\
-0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j \\
-0.2405+0.1388j \\
-0. +0.j \\
-0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j \\
0.0575-0.0996j \\
0. +0.j \\
0. +0.j
\end{bmatrix}
$$


### **Demostración matemática**

In [ ]:
q0 = estado_initial
q1 = estado_initial
q2 = [1, 0, 0]

estado_sist = np.kron(np.kron(q0, q1), q2)
print('Estado del sistema (3 qutrits):')
print(np.round(estado_sist,4))
print('Dimensión del estado del sistema: ', estado_sist.shape)

Estado del sistema (3 qutrits):
[ 0.6704-0.j      0.    +0.j      0.    +0.j     -0.0982-0.3664j
  0.    -0.j      0.    -0.j     -0.2405+0.1388j -0.    +0.j
 -0.    +0.j     -0.0982-0.3664j  0.    -0.j      0.    -0.j
 -0.1859+0.1073j -0.    +0.j     -0.    +0.j      0.1111+0.1111j
  0.    +0.j      0.    +0.j     -0.2405+0.1388j -0.    +0.j
 -0.    +0.j      0.1111+0.1111j  0.    +0.j      0.    +0.j
  0.0575-0.0996j  0.    +0.j      0.    +0.j    ]
Dimensión del estado del sistema:  (27,)


### **Demostración práctica**

In [293]:

# --- Parámetros del circuito ---
num_particles = 2
wires = list(range(num_particles + 1))  # +1 ancilla
print('Información importante:')
print('-------------------------------------------------')
print('Numero de wires: ', len(wires))
print('Wires: ', wires)
ancilla = wires[-1]
print('Ancilla wire: ', ancilla)
print('-------------------------------------------------')

dev = qml.device("default.qutrit", wires=wires)  # Cambiado a qutrits
latent_wire = 0
trash_wires = wires[1:num_particles]
ref_wires = wires[num_particles:-1]

theta1 = torch.pi / torch.tensor(2)
theta2 = torch.pi / torch.tensor(4)
phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)


def TSWAP_matrix():
    tswap = np.zeros((9, 9), dtype=complex)
    for i in range(3):
        for j in range(3):
            ket = np.zeros(9)
            bra = np.zeros(9)
            ket[3*i + j] = 1   # |i⟩|j⟩
            bra[3*j + i] = 1   # |j⟩|i⟩
            tswap += np.outer(bra, ket)
    return tswap

# --- Encoder aproximado para qutrits ---
def encode_1p1q_qutrit():
    for i in range(num_particles):
        theta1 = torch.pi / torch.tensor(2)
        theta2 = torch.pi / torch.tensor(4)
        phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
        phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)
        initial_state = inicializing_qutrit_state(theta1, theta2, phi1, phi2)
        u = unitary_from_state(initial_state)
        qml.QutritUnitary(u, wires=i)
        

@qml.qnode(dev)
def qae_circuit_qutrit_1(num_layers):
    print('Paso 1: Encoding')
    print('-----------------------------------------------------')
    encode_1p1q_qutrit()
    print('-----------------------------------------------------')
    return qml.state()

np.round(qae_circuit_qutrit_1(4), 4)



Información importante:
-------------------------------------------------
Numero de wires:  3
Wires:  [0, 1, 2]
Ancilla wire:  2
-------------------------------------------------
Paso 1: Encoding
-----------------------------------------------------
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.31

array([ 0.6704-0.j    ,  0.    +0.j    ,  0.    +0.j    , -0.0982-0.3664j,
        0.    +0.j    ,  0.    +0.j    , -0.2405+0.1388j,  0.    +0.j    ,
        0.    +0.j    , -0.0982-0.3664j,  0.    +0.j    ,  0.    +0.j    ,
       -0.1859+0.1073j,  0.    +0.j    ,  0.    +0.j    ,  0.1111+0.1111j,
        0.    +0.j    ,  0.    +0.j    , -0.2405+0.1388j,  0.    +0.j    ,
        0.    +0.j    ,  0.1111+0.1111j,  0.    +0.j    ,  0.    +0.j    ,
        0.0575-0.0996j,  0.    +0.j    ,  0.    +0.j    ])


$$

\ket{psi}^{\text{Matematicamente}}= \begin{bmatrix}
0.6704-0.j \\
0. +0.j \\
0. +0.j \\
-0.0982-0.3664j \\
0. -0.j \\
0. -0.j \\
-0.2405+0.1388j \\
-0. +0.j \\
-0. +0.j \\
-0.0982-0.3664j \\
0. -0.j \\
0. -0.j \\
-0.1859+0.1073j \\
-0. +0.j \\
-0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j \\
-0.2405+0.1388j \\
-0. +0.j \\
-0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j \\
0.0575-0.0996j \\
0. +0.j \\
0. +0.j
\end{bmatrix}, \quad
\ket{\psi}^{\text{Pennylane}} = \begin{bmatrix}
0.6704-0.j \\
0. +0.j \\
0. +0.j \\
-0.0982-0.3664j \\
0. +0.j \\
0. +0.j \\
-0.2405+0.1388j \\
0. +0.j \\
0. +0.j \\
-0.0982-0.3664j \\
0. +0.j \\
0. +0.j \\
-0.1859+0.1073j \\
0. +0.j \\
0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j \\
-0.2405+0.1388j \\
0. +0.j \\
0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j \\
0.0575-0.0996j \\
0. +0.j \\
0. +0.j
\end{bmatrix}, \quad
\text{Dimensión: } (27)
$$


Perfectísimo!!!

---

## **Paso 2. Variational layer**

Aplicamos ahora la puerta **TAdd** sobre lso qutrits 0 y 1 teniendo el control en el 0:

Palabras textuales de Pennylane:

*The 2-qutrit controlled add gate
The construction of this operator is based on definition 7 from Yeh et al. (2022). It performs the controlled TShift operation, and sends TAdd|𝑖⟩|𝑗⟩=|𝑖⟩|𝑖+𝑗⟩, where addition is taken modulo 3. The matrix representation is*

$$
\text{TAdd} =
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0
\end{bmatrix}
$$

$$
\text{Estado del sistema (2 qutrits, 0 y 1):} \\
\begin{bmatrix}
0.6704-0.j \\
-0.0982-0.3664j \\
-0.2405+0.1388j \\
-0.0982-0.3664j \\
-0.1859+0.1073j \\
0.1111+0.1111j \\
-0.2405+0.1388j \\
0.1111+0.1111j \\
0.0575-0.0996j
\end{bmatrix}, \quad
\text{Dimensión: } (9)
$$

$$
\text{Estado del sistema tras TAdd (2 qutrits, 0 y 1):} \\

\begin{bmatrix}
0.6704-0.j \\
-0.0982-0.3664j \\
-0.2405+0.1388j \\
-0.1859+0.1073j \\
0.1111+0.1111j \\
-0.0982-0.3664j \\
0.0575-0.0996j \\
-0.2405+0.1388j \\
0.1111+0.1111j
\end{bmatrix}
$$

$$
\text{Estado del sistema (3 qutrits):} \\

\begin{bmatrix}
0.6704-0.j \\
0. +0.j \\
0. +0.j \\
-0.0982-0.3664j \\
0. -0.j \\
0. -0.j \\
-0.2405+0.1388j \\
-0. +0.j \\
-0. +0.j \\
-0.1859+0.1073j \\
-0. +0.j \\
-0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j \\
-0.0982-0.3664j \\
0. -0.j \\
0. -0.j \\
0.0575-0.0996j \\
0. +0.j \\
0. +0.j \\
-0.2405+0.1388j \\
-0. +0.j \\
-0. +0.j \\
0.1111+0.1111j \\
0. +0.j \\
0. +0.j
\end{bmatrix}, \quad
\text{Dimensión: } (27)
$$


In [297]:
q0 = estado_initial
q1 = estado_initial
q2 = [1, 0, 0]
TAdd = qml.TAdd.compute_matrix()


estado_sist_01 = np.kron( q0, q1)
print('Estado del sistema (2 qutrits (el 0 y el 1)):')
print(np.round(estado_sist_01,4))
print('Dimensión del estado del sistema reducido: ', estado_sist_01.shape)
print('-----------------------------------------------------')
estado_sist_01 =  TAdd @ estado_sist_01
print('Estado del sistema tras  TAdd (2 qutrits (el 0 y el 1)):')
print(np.round(estado_sist_01,4))
print('-----------------------------------------------------')

Estado del sistema (2 qutrits (el 0 y el 1)):
[ 0.6704-0.j     -0.0982-0.3664j -0.2405+0.1388j -0.0982-0.3664j
 -0.1859+0.1073j  0.1111+0.1111j -0.2405+0.1388j  0.1111+0.1111j
  0.0575-0.0996j]
Dimensión del estado del sistema reducido:  (9,)
-----------------------------------------------------
Estado del sistema tras  TAdd (2 qutrits (el 0 y el 1)):
[ 0.6704-0.j     -0.0982-0.3664j -0.2405+0.1388j  0.1111+0.1111j
 -0.0982-0.3664j -0.1859+0.1073j  0.1111+0.1111j  0.0575-0.0996j
 -0.2405+0.1388j]
-----------------------------------------------------


In [298]:
print()
print()
print('------------------------------------------------------')
print('-----------------------------------------------------')
print('Aplicamos ahora las rotaciones a los estados recuperados')

theta1 = torch.pi / torch.tensor(2)
theta2 = torch.pi / torch.tensor(4)
phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)


u_x = unitary_from_generator(Sigma[1], theta1)
u_y = unitary_from_generator(Sigma[2], theta2)
u_z = unitary_from_generator(Sigma[3], phi2)
RotX_q0 = np.kron(u_x, np.eye(3))
RotY_q0 = np.kron(u_y, np.eye(3))
RotZ_q0 = np.kron(u_z, np.eye(3))

RotX_q1 = np.kron(np.eye(3), u_x)
RotY_q1 = np.kron(np.eye(3), u_y)
RotZ_q1 = np.kron(np.eye(3), u_z)
print('-----------------------------------------------------')
print('Primero en el qutrit 0:')
estado_rotado_mat = RotX_q0 @ estado_sist_01
estado_rotado_mat = RotY_q0 @ estado_rotado_mat
estado_rotado_mat = RotZ_q0 @ estado_rotado_mat
print('-----------------------------------------------------')
print('Segundo en el qutrit 1:')    
estado_rotado_mat = RotX_q1 @ estado_rotado_mat
estado_rotado_mat = RotY_q1 @ estado_rotado_mat
estado_rotado_mat = RotZ_q1 @ estado_rotado_mat
print('-----------------------------------------------------')
print('Qutrits rotado:')
print(np.round(estado_rotado_mat,4))

print('-----------------------------------------------------')
print('Estado total del sistema:')
estado_sist = np.kron(estado_rotado_mat, q2)
print(np.round(estado_sist,4))
print('Dimensión del estado del sistema: ', estado_sist.shape)



------------------------------------------------------
-----------------------------------------------------
Aplicamos ahora las rotaciones a los estados recuperados
-----------------------------------------------------
Primero en el qutrit 0:
-----------------------------------------------------
Segundo en el qutrit 1:
-----------------------------------------------------
Qutrits rotado:
[-0.4913-0.0403j -0.2409+0.2939j  0.0411+0.0759j  0.3665+0.0449j
 -0.0047-0.5804j -0.1563-0.0184j -0.0771-0.04j   -0.0871+0.1686j
  0.2068-0.1442j]
-----------------------------------------------------
Estado total del sistema:
[-0.4913-0.0403j  0.    -0.j      0.    -0.j     -0.2409+0.2939j
 -0.    +0.j     -0.    +0.j      0.0411+0.0759j  0.    +0.j
  0.    +0.j      0.3665+0.0449j  0.    +0.j      0.    +0.j
 -0.0047-0.5804j  0.    -0.j      0.    -0.j     -0.1563-0.0184j
  0.    -0.j      0.    -0.j     -0.0771-0.04j    0.    -0.j
  0.    -0.j     -0.0871+0.1686j -0.    +0.j     -0.    +0.j
  0.

In [299]:
print(qml.TAdd.compute_matrix())

[[1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0]]


In [ ]:
# --- Parámetros del circuito ---
num_particles = 2
num_latent = 0
num_ref = 0
num_trash = num_ref
wires = list(range(num_particles + num_ref + 1))  # +1 ancilla
print('Información importante:')
print('-------------------------------------------------')
print('Numero de wires: ', len(wires))
print('Wires: ', wires)
ancilla = wires[-1]
print('Ancilla wire: ', ancilla)
print('-------------------------------------------------')

dev = qml.device("default.qutrit", wires=wires)  # Cambiado a qutrits
latent_wire = 0
trash_wires = wires[1:num_particles]
ref_wires = wires[num_particles:-1]


theta1 = torch.pi / torch.tensor(2)
theta2 = torch.pi / torch.tensor(4)
phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)



# --- Encoder aproximado para qutrits ---
def encode_1p1q_qutrit():
    
    for i in range(num_particles):
        theta1 = torch.pi / torch.tensor(2)
        theta2 = torch.pi / torch.tensor(4)
        phi1 = torch.tensor(3) * torch.pi / torch.tensor(2)
        phi2 = torch.tensor(4) * torch.pi / torch.tensor(3)
        initial_state = inicializing_qutrit_state(theta1, theta2, phi1, phi2)
        u = unitary_from_state(initial_state)
        qml.QutritUnitary(u, wires=i)
        
def variational_layer_qutrit():
    
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            qml.TAdd(wires=[i, j])
    for i in range(num_particles):

        RX = unitary_from_generator(Sigma[1], theta1)
        RY = unitary_from_generator(Sigma[2], theta2)
        RZ = unitary_from_generator(Sigma[3], phi2)

        qml.QutritUnitary(RX, wires=i)
        qml.QutritUnitary(RY, wires=i) 
        qml.QutritUnitary(RZ, wires=i)

@qml.qnode(dev)
def qae_circuit_qutrit_1():
    print('Paso 1: Encoding')
    print('-----------------------------------------------------')
    encode_1p1q_qutrit()
    print('-----------------------------------------------------')
    print('Paso 2: Variational Layer')
    variational_layer_qutrit()
    print('-----------------------------------------------------')
    return qml.state()

np.round(qae_circuit_qutrit_1().clone().detach().numpy(), 4)



Información importante:
-------------------------------------------------
Numero de wires:  3
Wires:  [0, 1, 2]
Ancilla wire:  2
-------------------------------------------------
Paso 1: Encoding
-----------------------------------------------------
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.31

array([-0.4913-0.0403j,  0.    +0.j    ,  0.    +0.j    , -0.2409+0.2939j,
        0.    +0.j    ,  0.    +0.j    ,  0.0411+0.0759j,  0.    +0.j    ,
        0.    +0.j    ,  0.3665+0.0449j,  0.    +0.j    ,  0.    +0.j    ,
       -0.0047-0.5804j,  0.    +0.j    ,  0.    +0.j    , -0.1563-0.0184j,
        0.    +0.j    ,  0.    +0.j    , -0.0771-0.04j  ,  0.    +0.j    ,
        0.    +0.j    , -0.0871+0.1686j,  0.    +0.j    ,  0.    +0.j    ,
        0.2068-0.1442j,  0.    +0.j    ,  0.    +0.j    ])

In [305]:
print('Caracteristicas del resultado: ')
st = qae_circuit_qutrit_1().clone().detach().numpy()
print('Tamaño: ', st.shape)

Caracteristicas del resultado: 
Paso 1: Encoding
-----------------------------------------------------
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
---------------------------------------

El resultado de las operaciones anteriores es: 

$$
\ket{\psi}^{\text{Pennylane}}_{\text{Variational}} =
\begin{bmatrix}
-0.4913 - 0.0403 i \\
0 \\
0\\
-0.2409 + 0.2939 i \\
0\\
0 \\
0.0411 + 0.0759 i \\
0\\
0\\
0.3665 + 0.0449 i \\
0\\
0\\
-0.0047 - 0.5804 i \\
0\\
0\\
-0.1563 - 0.0184 i \\
0\\
0. \\
-0.0771 - 0.0400 i \\
0\\
0\\
-0.0871 + 0.1686 i \\
0 \\
0\\
0.2068 - 0.1442 i \\
0\\
0
\end{bmatrix} \quad
\ket{\psi}^{\text{Matematicamente}}_{\text{Variational}}  = 
\begin{bmatrix}
-0.4913 - 0.0403 i \\
0 \\
0 \\
-0.2409 + 0.2939 i \\
0 \\
0 \\
0.0411 + 0.0759 i \\
0 \\
0 \\
0.3665 + 0.0449 i \\
0 \\
0 \\
-0.0047 - 0.5804 i \\
0 \\
0 \\
-0.1563 - 0.0184 i \\
0 \\
0 \\
-0.0771 - 0.0400 i \\
0 \\
0 \\
-0.0871 + 0.1686 i \\
0 \\
0 \\
0.2068 - 0.1442 i \\
0 \\
0
\end{bmatrix}
$$


---

### **Paso 3: Test SWAP**

Respecto **qml.probs(wires=ancilla)**:


Probability of each computational basis state.
This measurement function accepts either a wire specification or an observable. Passing wires to the function instructs the QNode to return a flat array containing the probabilities |⟨𝑖|𝜓⟩|2 of measuring the computational basis state |𝑖⟩ given the current state |𝜓⟩.



In [309]:
print('-----------------------------------------------------')
@qml.qnode(dev, interface="torch", diff_method="backprop")
def qae_circuit_qutrit():
    print('Paso 1: Encoding')
    print('-----------------------------------------------------')
    encode_1p1q_qutrit()
    print('-----------------------------------------------------')
    print('Paso 2: Variational Layer')
    variational_layer_qutrit()
    print('-----------------------------------------------------')
    print('Paso 3: TSWAP test')
    tswap = TSWAP_matrix()
    print('-----------------------------------------------------')

    for trash_wire, ref_wire in zip(trash_wires, ref_wires):
        qml.THadamard(wires=ancilla, subspace=None) # subspace=None aplica a todo el qutrit
        qml.ControlledQutritUnitary(tswap, control_wires=ancilla, wires=[trash_wire, ref_wire])
        qml.THadamard(wires=ancilla, subspace=None)
    #print('Probabilidades ancilla: ', qml.probs(wires=ancilla))
    return qml.probs(wires=ancilla)
    
def cost_function_with_fidelity_qutrit():
    probs = qae_circuit_qutrit()
    fidelity = probs[0]
    return -fidelity, fidelity.item(), probs

loss, fidelity, probs = cost_function_with_fidelity_qutrit()

-----------------------------------------------------
Paso 1: Encoding
-----------------------------------------------------
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
Gamma: tensor(0.7441)
State: tensor([ 0.6874+0.0000j, -0.1007-0.3757j, -0.2466+0.1424j],
       dtype=torch.complex128)
Norma:  tensor(0.8396, dtype=torch.float64)
Estado normalizado: tensor([ 0.8188+0.0000j, -0.1199-0.4475j, -0.2937+0.1696j],
       dtype=torch.complex128)
[[-0.8188+0.j     -0.4378-0.1066j  0.1654-0.315j ]
 [ 0.1199+0.4475j -0.1066-0.1648j  0.8048+0.315j ]
 [ 0.2937-0.1696j -0.8643+0.1066j -0.1654+0.315j ]]
-----------------

In [ ]:
print('Fidelity: ', fidelity) 
print('Probabilidades ancilla: ', probs.clone().detach().numpy())


Fidelity:  0.811726000682539
Probabilidades ancilla:  [0.811726   0.09413693 0.09413693]
